## <b><font color='darkblue'>Preface</font></b>
([source](https://learn.deeplearning.ai/courses/langchain/lesson/7/agents)) <b><font size='3ptx'>The core idea of agents is to use a language model to choose a sequence of actions to take. ([more](https://python.langchain.com/v0.1/docs/modules/agents/))</font></b>

In chains, a sequence of actions is hardcoded (in code). <b>In agents, a language model is used as a reasoning engine to determine which actions to take and in which order</b>.

### <b><font color='darkgreen'>Outline</font></b>
* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [8]:
!pip freeze | grep -P '(openai|langchain)'

langchain==0.2.6
langchain-anthropic==0.1.15
langchain-community==0.2.6
langchain-core==0.2.10
langchain-experimental==0.0.62
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
openai==1.28.1


In [9]:
import datetime
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
import pandas as pd


TEST_DATA = pd.DataFrame({'index': ['row1', 'row2'], 'review': ['review1', 'review2']})

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [10]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## <b><font color='darkblue'>Built-in LangChain tools</font></b>

In [3]:
#!pip install -U wikipedia

In [78]:
import langchain
from langchain_experimental.agents.agent_toolkits.python.base import create_python_agent
from langchain.agents import load_tools, initialize_agent, create_react_agent, AgentExecutor
#from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import AgentType
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.utilities.python import PythonREPL
from langchain_openai import ChatOpenAI
from langchain import hub

In [15]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [54]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm)

In [69]:
tools[0].__class__

langchain_core.tools.Tool

In [26]:
prompt = hub.pull("hwchase17/react")

In [41]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [37]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [42]:
agent_executor.invoke({'input': "What is the result of equation of '1 + 5'?"})



> Entering new AgentExecutor chain...
I should use the Calculator tool to solve this math equation.
Action: Calculator
Action Input: 1 + 5Answer: 6I now know the final answer
Final Answer: 6

> Finished chain.


{'input': "What is the result of equation of '1 + 5'?", 'output': '6'}

### <b><font color='darkgreen'>Wikipedia example</font></b>

In [43]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"

In [44]:
agent_executor.invoke({'input': question})



> Entering new AgentExecutor chain...
I should use Wikipedia to find out what book Tom M. Mitchell wrote.
Action: wikipedia
Action Input: Tom M. Mitchell

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footballer playing 

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Tom M. Mitchell wrote the textbook "Machine Learning."'}

### <b><font color='darkgreen'>Python Agent</font></b>

In [61]:
python_repl = PythonREPL()

In [62]:
python_repl.run("print(1+1)")

Python REPL can execute arbitrary code. Use with caution.


'2\n'

In [71]:
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
tools = [repl_tool]

In [81]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [82]:
#agent_executor = AgentExecutor(
#    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [74]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [75]:
question2 = f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}"""

In [84]:
langchain.debug=True
resp = agent.run(question2)
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

In [85]:
resp

"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

## <b><font color='darkblue'>Define your own tool</font></b>
<b><font size='3ptx'>When constructing your own agent, you will need to provide it with a list of Tools that it can use. ([more](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/))</font></b>

Besides the actual function that is called, the Tool consists of several components:
* **`name` (str)**, is required and must be unique within a set of tools provided to an agent
* **`description` (str)**, is optional but recommended, as it is used by an agent to determine tool use
* **`args_schema` (Pydantic BaseModel)**, is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.

In [96]:
from langchain.agents import tool
from datetime import date

In [97]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [90]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm) + [time]

In [91]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). If it does, please try running it again.

In [99]:
try:
    result = agent_executor.invoke({'input': "What date is of today?"})
except Exception as ex: 
    print(f"Exception on external access: {ex}")



> Entering new AgentExecutor chain...
I should use the time function to get today's date.
Action: time
Action Input: ""2024-06-27I have the current date.
Final Answer: 2024-06-27

> Finished chain.


## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearning.AI - Langchain Ch6 - Evaluation](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch6_evaluation.ipynb)
* [[LangChain for LLM Application Development] 課程筆記- Agents](https://hackmd.io/@YungHuiHsu/rkBMDgRM6)